<a href="https://colab.research.google.com/github/amandatz/linear-programming/blob/main/Atividade1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade 1



Amanda Topanotti Zanette (22100776)

**Importações e funções auxiliares**

In [ ]:
using LinearAlgebra, Printf

print_matrix (generic function with 1 method)

Esse trecho de código é usado apenas para garantir a repetibilidade dos números aleatórios gerados.

In [ ]:
using Random
Random.seed!(123)

## Questão 1

## Questão 2

## Questão 3